In [20]:
one_hot_X, wt = sess.run(
        [outputs, weights], feed_dict={X: dataX, Y: dataY})

one_hot_X

wt

array([[ 1.,  1.,  1., ...,  1.,  1.,  1.],
       [ 1.,  1.,  1., ...,  1.,  1.,  1.],
       [ 1.,  1.,  1., ...,  1.,  1.,  1.],
       ..., 
       [ 1.,  1.,  1., ...,  1.,  1.,  1.],
       [ 1.,  1.,  1., ...,  1.,  1.,  1.],
       [ 1.,  1.,  1., ...,  1.,  1.,  1.]], dtype=float32)

In [24]:
print(wt.shape, one_hot_X.shape)

(170, 10) (170, 10, 25)


In [1]:
from __future__ import print_function

import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn

tf.set_random_seed(777)  # reproducibility

sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

char_set = list(set(sentence))
char_dic = {w: i for i, w in enumerate(char_set)}

data_dim = len(char_set)
hidden_size = len(char_set)
num_classes = len(char_set)
sequence_length = 10  # Any arbitrary number
learning_rate = 0.1

dataX = []
dataY = []
for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    print(i, x_str, '->', y_str)

    x = [char_dic[c] for c in x_str]  # x str to index
    y = [char_dic[c] for c in y_str]  # y str to index

    dataX.append(x)
    dataY.append(y)

batch_size = len(dataX)


X = tf.placeholder(tf.int32, [None, sequence_length])
Y = tf.placeholder(tf.int32, [None, sequence_length])

# One-hot encoding
X_one_hot = tf.one_hot(X, num_classes)
print(X_one_hot)  # check out the shape


# Make a lstm cell with hidden_size (each unit output vector size)
def lstm_cell():
    '''
    __init__(
        num_units,
        forget_bias=1.0,
        input_size=None,
        state_is_tuple=True,
        activation=tf.tanh,
        reuse=None
    )
    '''
    cell = rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
    return cell

multi_cells = rnn.MultiRNNCell([lstm_cell() for _ in range(2)], state_is_tuple=True)


'''
X: seq_length
Y: seq_length

X_one_hot: seq_length * num_classes 
'''
# outputs: unfolding size x hidden size, state = hidden size
outputs, _states = tf.nn.dynamic_rnn(multi_cells, X_one_hot, dtype=tf.float32)

# FC layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)

# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

# All weights are 1 (equal weights)
weights = tf.ones([batch_size, sequence_length])

sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
mean_loss = tf.reduce_mean(sequence_loss)
train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(mean_loss)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(500):
    _, l, results = sess.run(
        [train_op, mean_loss, outputs], feed_dict={X: dataX, Y: dataY})
    for j, result in enumerate(results):
        index = np.argmax(result, axis=1)
        print(i, j, ''.join([char_set[t] for t in index]), l)

# Let's print the last char of each result to check it works
results = sess.run(outputs, feed_dict={X: dataX})
for j, result in enumerate(results):
    index = np.argmax(result, axis=1)
    if j is 0:  # print all for the first result to make a sentence
        print(''.join([char_set[t] for t in index]), end='')
    else:
        print(char_set[index[-1]], end='')

'''
0 167 tttttttttt 3.23111
0 168 tttttttttt 3.23111
0 169 tttttttttt 3.23111
…
499 167  of the se 0.229616
499 168 tf the sea 0.229616
499 169   the sea. 0.229616

g you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.

'''


0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

2 164            3.17223
2 165            3.17223
2 166            3.17223
2 167            3.17223
2 168            3.17223
2 169            3.17223
3 0 eeeeeeeeee 2.94585
3 1 eeeeeeeeee 2.94585
3 2 eeeeeeeeee 2.94585
3 3 eeeeeeeeee 2.94585
3 4 eeeeeeeeee 2.94585
3 5 eeeeeeeeee 2.94585
3 6 eeeeeeeeee 2.94585
3 7 eeeeeeeeee 2.94585
3 8 eeeeeeeeee 2.94585
3 9 eeeeeeeeee 2.94585
3 10 eeeeeeeeee 2.94585
3 11 eeeeeeeeee 2.94585
3 12 eeeeeeeeee 2.94585
3 13 eeeeeeeeee 2.94585
3 14 eeeeeeeeee 2.94585
3 15 eeeeeeeeee 2.94585
3 16 eeeeeeeeee 2.94585
3 17 eeeeeeeeee 2.94585
3 18 eeeeeeeeee 2.94585
3 19 eeeeeeeeee 2.94585
3 20 eeeeeeeeee 2.94585
3 21 eeeeeeeeee 2.94585
3 22 eeeeeeeeee 2.94585
3 23 eeeeeeeeee 2.94585
3 24 eeeeeeeeee 2.94585
3 25 eeeeeeeeee 2.94585
3 26 eeeeeeeeee 2.94585
3 27 eeeeeeeeee 2.94585
3 28 eeeeeeeeee 2.94585
3 29 eeeeeeeeee 2.94585
3 30 eeeeeeeeee 2.94585
3 31 eeeeeeeeee 2.94585
3 32 eeeeeeeeee 2.94585
3 33 eeeeeeeeee 2.94585
3 34 eeeeeeeeee 2.94585
3 35 eeeeeeeeee 2.94

5 169    o       2.90684
6 0   ooo  o   2.88199
6 1  ooo  o    2.88199
6 2 ooo oo     2.88199
6 3     o      2.88199
6 4    o       2.88199
6 5   oo   o   2.88199
6 6 ooo   o    2.88199
6 7      o     2.88199
6 8     o      2.88199
6 9   oooo     2.88199
6 10  oooo      2.88199
6 11 oooo       2.88199
6 12   o        2.88199
6 13            2.88199
6 14 o          2.88199
6 15            2.88199
6 16     o      2.88199
6 17    o       2.88199
6 18   o o      2.88199
6 19  o o       2.88199
6 20 o o        2.88199
6 21            2.88199
6 22 o          2.88199
6 23            2.88199
6 24     o      2.88199
6 25    oo      2.88199
6 26   ooo      2.88199
6 27            2.88199
6 28 ooo        2.88199
6 29            2.88199
6 30            2.88199
6 31            2.88199
6 32   o        2.88199
6 33  oo   o oo 2.88199
6 34 oo   o ooo 2.88199
6 35     o oooo 2.88199
6 36      ooooo 2.88199
6 37   o oooooo 2.88199
6 38  o ooooooo 2.88199
6 39 o oooooooo 2.88199
6 40   oooooooo 2.88199
6

8 108            2.82482
8 109            2.82482
8 110            2.82482
8 111            2.82482
8 112            2.82482
8 113            2.82482
8 114            2.82482
8 115            2.82482
8 116            2.82482
8 117            2.82482
8 118            2.82482
8 119            2.82482
8 120            2.82482
8 121            2.82482
8 122            2.82482
8 123            2.82482
8 124            2.82482
8 125            2.82482
8 126            2.82482
8 127            2.82482
8 128            2.82482
8 129            2.82482
8 130            2.82482
8 131            2.82482
8 132            2.82482
8 133            2.82482
8 134            2.82482
8 135            2.82482
8 136            2.82482
8 137            2.82482
8 138            2.82482
8 139            2.82482
8 140            2.82482
8 141            2.82482
8 142            2.82482
8 143            2.82482
8 144            2.82482
8 145            2.82482
8 146            2.82482
8 147            2.82482


11 77            2.7265
11 78            2.7265
11 79            2.7265
11 80            2.7265
11 81            2.7265
11 82            2.7265
11 83            2.7265
11 84            2.7265
11 85            2.7265
11 86            2.7265
11 87            2.7265
11 88            2.7265
11 89            2.7265
11 90            2.7265
11 91            2.7265
11 92            2.7265
11 93            2.7265
11 94            2.7265
11 95            2.7265
11 96            2.7265
11 97            2.7265
11 98            2.7265
11 99            2.7265
11 100            2.7265
11 101            2.7265
11 102            2.7265
11 103            2.7265
11 104            2.7265
11 105            2.7265
11 106            2.7265
11 107            2.7265
11 108            2.7265
11 109            2.7265
11 110            2.7265
11 111            2.7265
11 112            2.7265
11 113            2.7265
11 114            2.7265
11 115            2.7265
11 116            2.7265
11 117            2.726

...

489 79  n't assig 0.228877
489 80  't assign 0.228877
489 81 dt dssign  0.228877
489 82 t dssign t 0.228877
489 83 hwssign th 0.228877
489 84 tnsign the 0.228877
489 85 nkign them 0.228877
489 86 iign them  0.228877
489 87 itn them t 0.228877
489 88 mn them ta 0.228877
489 89   them tas 0.228877
489 90 dthem task 0.228877
489 91 toem tosks 0.228877
489 92 her tosks  0.228877
489 93 er tosks a 0.228877
489 94 n tosks an 0.228877
489 95  tosks and 0.228877
489 96 tosks and  0.228877
489 97 hsks and w 0.228877
489 98 nks and wo 0.228877
489 99 is and wor 0.228877
489 100 , and work 0.228877
489 101 iind work, 0.228877
489 102 tnd work,  0.228877
489 103 nd dork, b 0.228877
489 104 d dork, bu 0.228877
489 105  aork, but 0.228877
489 106 took, but  0.228877
489 107 ook, but r 0.228877
489 108   , but ra 0.228877
489 109  , but rat 0.228877
489 110 , but rath 0.228877
489 111  dut rathe 0.228877
489 112 tui rather 0.228877
489 113 ut rather  0.228877
489 114 t rather t 0.228877
489 115 hwath

493 0 m you want 0.228871
493 1  you want  0.228871
493 2 tou want t 0.228871
493 3 ou want to 0.228871
493 4   want to  0.228871
493 5 twant to b 0.228871
493 6 tont to bu 0.228871
493 7 ont to bui 0.228871
493 8 nd to buil 0.228871
493 9 d to build 0.228871
493 10 hwo build  0.228871
493 11 to build a 0.228871
493 12 h build a  0.228871
493 13  build a s 0.228871
493 14 tuild a sh 0.228871
493 15 utld a shi 0.228871
493 16 tld a ship 0.228871
493 17 md a ship, 0.228871
493 18 e a ship,  0.228871
493 19  a ship, d 0.228871
493 20 tnship, do 0.228871
493 21 nship, don 0.228871
493 22 thip, don' 0.228871
493 23 iip, don't 0.228871
493 24 ep, don't  0.228871
493 25 m, don't d 0.228871
493 26 l don't dr 0.228871
493 27  don't dru 0.228871
493 28 ton't arum 0.228871
493 29  n't arum  0.228871
493 30  't drum u 0.228871
493 31 dt drum up 0.228871
493 32 t arum up  0.228871
493 33 hwrum up p 0.228871
493 34 toum up pe 0.228871
493 35  um up peo 0.228871
493 36  m up peop 0.228871
493 37 t up

495 131 em ta long 0.228869
495 132 n to long  0.228869
495 133  to long f 0.228869
495 134 to bong fo 0.228869
495 135   bong for 0.228869
495 136 nlong for  0.228869
495 137 tong for t 0.228869
495 138 eng for th 0.228869
495 139 n' for the 0.228869
495 140 d for the  0.228869
495 141  for the e 0.228869
495 142 tor the en 0.228869
495 143  r the end 0.228869
495 144 n the endl 0.228869
495 145  toe endle 0.228869
495 146 toemendles 0.228869
495 147  emendless 0.228869
495 148 emendless  0.228869
495 149 nendless i 0.228869
495 150 tndless im 0.228869
495 151 nsless imm 0.228869
495 152 d ess imme 0.228869
495 153  ess immen 0.228869
495 154 ess immens 0.228869
495 155 ns immensi 0.228869
495 156 iiimmensit 0.228869
495 157 iimmensity 0.228869
495 158 tmmensity  0.228869
495 159 mmensity o 0.228869
495 160  ensity of 0.228869
495 161  nsity of  0.228869
495 162 nsity of t 0.228869
495 163 dity of th 0.228869
495 164 igy of the 0.228869
495 165 my of the  0.228869
495 166   of the s 0

498 117  ther teac 0.228865
498 118 nher teach 0.228865
498 119 her teach  0.228865
498 120 er toach t 0.228865
498 121 r teach th 0.228865
498 122  thach the 0.228865
498 123 toach them 0.228865
498 124 hach them  0.228865
498 125 rch them t 0.228865
498 126 nh them to 0.228865
498 127 o them to  0.228865
498 128 ethem to l 0.228865
498 129 toem ta lo 0.228865
498 130 her to lon 0.228865
498 131 er ta long 0.228865
498 132 r ta long  0.228865
498 133  ta long f 0.228865
498 134 to long fo 0.228865
498 135 h bong for 0.228865
498 136  long for  0.228865
498 137 tong for t 0.228865
498 138 eng for th 0.228865
498 139  ' for the 0.228865
498 140 d for the  0.228865
498 141  for the e 0.228865
498 142 tor the en 0.228865
498 143  r the end 0.228865
498 144   the endl 0.228865
498 145  the endle 0.228865
498 146 toemendles 0.228865
498 147 herendless 0.228865
498 148 erendless  0.228865
498 149 rtndless i 0.228865
498 150 tndless im 0.228865
498 151 rsless imm 0.228865
498 152 d ess imme 0

"\n0 167 tttttttttt 3.23111\n0 168 tttttttttt 3.23111\n0 169 tttttttttt 3.23111\n…\n499 167  of the se 0.229616\n499 168 tf the sea 0.229616\n499 169   the sea. 0.229616\n\ng you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.\n\n"